In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

### PD Set Options

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [4]:
dsf1 = tabula.read_pdf('Topcondoimage 11-07-2022 (2).pdf', pages='all',lattice=True)
dsf2 = tabula.read_pdf('Topcondoimage 11-14-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 11-21-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 11-28-2022 (2).pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 12-05-2022 (2).pdf', pages='all',lattice=True)

In [6]:
df1 = dsf1[0]
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]

In [7]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]

In [8]:
df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])

In [9]:
df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]

In [10]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [11]:
# df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [12]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [13]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
0,Estates at Aqualina\r17901 Collins Ave 4207\rSunny Isles Beach,6/30/2022,11/1/2022,42,"$11,000,000.00",5573,"$1,973.80",Marla Cohen,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Aqualina 17901 Collins Ave 4207 Sunny Isles Beach,Estates at Aqualina,17901 Collins Ave 4207 Sunny Isles Beach
1,Trump Palace\r18101 Collins Ave 1809\rSunny Isles Beach,9/1/2022,11/1/2022,39,"$3,000,000.00",2681,"$1,232.03",Lana Bell,Douglas Elliman,Patricia Hachar,"Compass Florida, LLC.",Trump Palace 18101 Collins Ave 1809 Sunny Isles Beach,Trump Palace,18101 Collins Ave 1809 Sunny Isles Beach
2,Setai\r101 20th St 3101\rMiami Beach,6/16/2022,10/31/2022,117,"$2,850,000.00",1198,"$2,378.96",Diana Garchitorena,Douglas Elliman,Devin Kay,Douglas Elliman,Setai 101 20th St 3101 Miami Beach,Setai,101 20th St 3101 Miami Beach
3,1 Hotel & Homes\r102 24th St 1114\rMiami Beach,10/7/2022,10/31/2022,1,"$2,550,000.00",1084,"$2,628.87",Tracy Galya,Douglas Elliman,Tessa Trlaja,COSMORE Florida Corp.,1 Hotel & Homes 102 24th St 1114 Miami Beach,,1 Hotel & Homes 102 24th St 1114 Miami Beach
4,Jade Beach\r17001 Collins Ave 1704\rSunny Isles Beach,10/10/2022,11/2/2022,6,"$2,450,000.00",1927,"$1,271.41",Marcello Agostini,"AG Real Estate Advisors, LLC",Marina Petlyakova,Golden Keys Inc,Jade Beach 17001 Collins Ave 1704 Sunny Isles Beach,Jade Beach,17001 Collins Ave 1704 Sunny Isles Beach
5,The Ocean Club\r789 Crandon Blvd 1402\rKey Biscayne,9/6/2022,10/31/2022,13,"$2,395,000.00",2100,"$1,140.48",Douglas Kinsley,Fortune Int'l Realty Key Biscayne Inc,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",The Ocean Club 789 Crandon Blvd 1402 Key Biscayne,The Ocean Club,789 Crandon Blvd 1402 Key Biscayne
6,Majestic\r9601 Collins Ave 301\rBal Harbour,9/29/2022,11/4/2022,11,"$2,000,000.00",2260,$884.96,Shneur Shapira,The Brokerage South Florida Real Estate,Joel Lusky,The Brokerage South Florida Real Estate,Majestic 9601 Collins Ave 301 Bal Harbour,Majestic,9601 Collins Ave 301 Bal Harbour
7,Bellini Williams Island\r4100 Island Blvd 1504\rAventura,7/25/2022,11/1/2022,77,"$1,900,000.00",2608,$728.53,Nicole Shein,"Williams Island Realty, LLC",Joel Matus,"Williams Island Realty, LLC",Bellini Williams Island 4100 Island Blvd 1504 Aventura,Bellini Williams Island,4100 Island Blvd 1504 Aventura
8,The Meridian Condo\r2001 Meridian Ave 307\rMiami Beach,5/11/2022,10/31/2022,128,"$1,722,500.00",2058,$836.98,Sebastian Wagner,"Brokers, LLC",Maurice Boschetti,"Boschetti Realty Group, LLC.",The Meridian Condo 2001 Meridian Ave 307 Miami Beach,The Meridian Condo,2001 Meridian Ave 307 Miami Beach
9,M0 arina Palms Residences\r17301 Biscayne Blvd 809\rNorth Miami Beach,5/13/2022,10/31/2022,158,"$1,705,000.00",2219,$768.36,Tania Weiss,Fortune International Realty,Marc Plotkin,Coldwell Banker Realty,M0 arina Palms Residences 17301 Biscayne Blvd 809 North Miami Beach,M,0 arina Palms Residences 17301 Biscayne Blvd 809 North Miami Beach


In [14]:
x = df['Close Date'].str.startswith('11')
df = df[x]

In [15]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
0,Estates at Aqualina\r17901 Collins Ave 4207\rSunny Isles Beach,6/30/2022,11/1/2022,42,"$11,000,000.00",5573,"$1,973.80",Marla Cohen,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Aqualina 17901 Collins Ave 4207 Sunny Isles Beach,Estates at Aqualina,17901 Collins Ave 4207 Sunny Isles Beach
1,Trump Palace\r18101 Collins Ave 1809\rSunny Isles Beach,9/1/2022,11/1/2022,39,"$3,000,000.00",2681,"$1,232.03",Lana Bell,Douglas Elliman,Patricia Hachar,"Compass Florida, LLC.",Trump Palace 18101 Collins Ave 1809 Sunny Isles Beach,Trump Palace,18101 Collins Ave 1809 Sunny Isles Beach
4,Jade Beach\r17001 Collins Ave 1704\rSunny Isles Beach,10/10/2022,11/2/2022,6,"$2,450,000.00",1927,"$1,271.41",Marcello Agostini,"AG Real Estate Advisors, LLC",Marina Petlyakova,Golden Keys Inc,Jade Beach 17001 Collins Ave 1704 Sunny Isles Beach,Jade Beach,17001 Collins Ave 1704 Sunny Isles Beach
6,Majestic\r9601 Collins Ave 301\rBal Harbour,9/29/2022,11/4/2022,11,"$2,000,000.00",2260,$884.96,Shneur Shapira,The Brokerage South Florida Real Estate,Joel Lusky,The Brokerage South Florida Real Estate,Majestic 9601 Collins Ave 301 Bal Harbour,Majestic,9601 Collins Ave 301 Bal Harbour
7,Bellini Williams Island\r4100 Island Blvd 1504\rAventura,7/25/2022,11/1/2022,77,"$1,900,000.00",2608,$728.53,Nicole Shein,"Williams Island Realty, LLC",Joel Matus,"Williams Island Realty, LLC",Bellini Williams Island 4100 Island Blvd 1504 Aventura,Bellini Williams Island,4100 Island Blvd 1504 Aventura
10,The Ritz Carlton Residences\r4701 N Meridian 221\rMiami Beach,NaN,11/9/2022,942,"$5,000,000.00",3265,"$1,531.39",Juan Carlos Pagan,HAUS Realty,JTHS Accounting,MIAMI Association of REALTORS®,The Ritz Carlton Residences 4701 N Meridian 221 Miami Beach,The Ritz Carlton Residences,4701 N Meridian 221 Miami Beach
11,Palau Sunset Harbour\r1201 20th St 302\rMiami Beach,NaN,11/7/2022,101,"$2,600,000.00",1728,"$1,504.63",Menachem Fellig,"Compass Florida, LLC.",John Sandberg,Douglas Elliman,Palau Sunset Harbour 1201 20th St 302 Miami Beach,Palau Sunset Harbour,1201 20th St 302 Miami Beach
12,Continuum South Tower\r100 S Pointe Dr 1403\rMiami Beach,NaN,11/7/2022,151,"$2,525,000.00",1365,"$1,849.82",Adam Amsterdam,LoKation,John Lennon,South Pointe Drive Realty Inc,Continuum South Tower 100 S Pointe Dr 1403 Miami Beach,Continuum South Tower,100 S Pointe Dr 1403 Miami Beach
13,Porto Vita\r20165 NE 39th Pl 401\rAventura,NaN,11/9/2022,89,"$2,300,000.00",3800,$605.26,Sylvia Fragos,Great Estates Int'l Realty,Stuart Berger,Coldwell Banker Realty,Porto Vita 20165 NE 39th Pl 401 Aventura,Porto Vita,20165 NE 39th Pl 401 Aventura
14,Grove Isle Condo\r2 Grove Isle Dr B902\rMiami,NaN,11/9/2022,78,"$1,500,000.00",2398,$625.52,David Sporn,Coldwell Banker Realty,Bethea Goldman,BHHS EWM Realty,Grove Isle Condo 2 Grove Isle Dr B902 Miami,Grove Isle Condo,2 Grove Isle Dr B902 Miami


In [16]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [17]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [18]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/22809927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [19]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/1957931448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find  coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [20]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [21]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/17039772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/17039772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/17039772.py:1: SettingWithCopyW

In [22]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [23]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
0,Estates at Aqualina\r17901 Collins Ave 4207\rSunny Isles Beach,6/30/2022,11/1/2022,42,"$11,000,000.00",5573,"$1,973.80",Marla Cohen,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Aqualina 17901 Collins Ave 4207 Sunny Isles Beach,Estates at Aqualina,17901 Collins Ave 4207 Sunny Isles Beach,"(17901 Collins Ave #4207, Sunny Isles Beach, FL 33160, USA, (25.9420489, -80.12019839999999))","(25.9420489, -80.12019839999999, 0.0)",25.942049,-80.120198,0.0
1,Trump Palace\r18101 Collins Ave 1809\rSunny Isles Beach,9/1/2022,11/1/2022,39,"$3,000,000.00",2681,"$1,232.03",Lana Bell,Douglas Elliman,Patricia Hachar,"Compass Florida, LLC.",Trump Palace 18101 Collins Ave 1809 Sunny Isles Beach,Trump Palace,18101 Collins Ave 1809 Sunny Isles Beach,"(18101 Collins Ave APT 1809, Sunny Isles Beach, FL 33160, USA, (25.9440252, -80.1202751))","(25.9440252, -80.1202751, 0.0)",25.944025,-80.120275,0.0
2,Jade Beach\r17001 Collins Ave 1704\rSunny Isles Beach,10/10/2022,11/2/2022,6,"$2,450,000.00",1927,"$1,271.41",Marcello Agostini,"AG Real Estate Advisors, LLC",Marina Petlyakova,Golden Keys Inc,Jade Beach 17001 Collins Ave 1704 Sunny Isles Beach,Jade Beach,17001 Collins Ave 1704 Sunny Isles Beach,"(17001 Collins Ave APT 1704, Sunny Isles Beach, FL 33160, USA, (25.9335244, -80.1211641))","(25.9335244, -80.1211641, 0.0)",25.933524,-80.121164,0.0
3,Majestic\r9601 Collins Ave 301\rBal Harbour,9/29/2022,11/4/2022,11,"$2,000,000.00",2260,$884.96,Shneur Shapira,The Brokerage South Florida Real Estate,Joel Lusky,The Brokerage South Florida Real Estate,Majestic 9601 Collins Ave 301 Bal Harbour,Majestic,9601 Collins Ave 301 Bal Harbour,"(9601 Collins Ave APT 301, Bal Harbour, FL 33154, USA, (25.8876563, -80.1222319))","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0
4,Bellini Williams Island\r4100 Island Blvd 1504\rAventura,7/25/2022,11/1/2022,77,"$1,900,000.00",2608,$728.53,Nicole Shein,"Williams Island Realty, LLC",Joel Matus,"Williams Island Realty, LLC",Bellini Williams Island 4100 Island Blvd 1504 Aventura,Bellini Williams Island,4100 Island Blvd 1504 Aventura,"(4100 Island Blvd #1504, Aventura, FL 33160, USA, (25.9410759, -80.1359681))","(25.9410759, -80.1359681, 0.0)",25.941076,-80.135968,0.0
5,The Ritz Carlton Residences\r4701 N Meridian 221\rMiami Beach,NaN,11/9/2022,942,"$5,000,000.00",3265,"$1,531.39",Juan Carlos Pagan,HAUS Realty,JTHS Accounting,MIAMI Association of REALTORS®,The Ritz Carlton Residences 4701 N Meridian 221 Miami Beach,The Ritz Carlton Residences,4701 N Meridian 221 Miami Beach,"(4701 Meridian Ave #221, Miami Beach, FL 33140, USA, (25.8230945, -80.1310154))","(25.8230945, -80.1310154, 0.0)",25.823094,-80.131015,0.0
6,Palau Sunset Harbour\r1201 20th St 302\rMiami Beach,NaN,11/7/2022,101,"$2,600,000.00",1728,"$1,504.63",Menachem Fellig,"Compass Florida, LLC.",John Sandberg,Douglas Elliman,Palau Sunset Harbour 1201 20th St 302 Miami Beach,Palau Sunset Harbour,1201 20th St 302 Miami Beach,"(1201 20th St Apt 302, Miami Beach, FL 33139, USA, (25.795997, -80.14206039999999))","(25.795997, -80.14206039999999, 0.0)",25.795997,-80.142060,0.0
7,Continuum South Tower\r100 S Pointe Dr 1403\rMiami Beach,NaN,11/7/2022,151,"$2,525,000.00",1365,"$1,849.82",Adam Amsterdam,LoKation,John Lennon,South Pointe Drive Realty Inc,Continuum South Tower 100 S Pointe Dr 1403 Miami Beach,Continuum South Tower,100 S Pointe Dr 1403 Miami Beach,"(100 South Pointe Dr APT 1403, Miami Beach, FL 33139, USA, (25.7676323, -80.137348))","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
8,Porto Vita\r20165 NE 39th Pl 401\rAventura,NaN,11/9/2022,89,"$2,300,000.00",3800,$605.26,Sylvia Fragos,Great Estates Int'l Realty,Stuart Berger,Coldwell Banker Realty,Porto Vita 20165 NE 39th Pl 401 Aventura,Porto Vita,20165 NE 39th Pl 401 Aventura,"(2

In [24]:
df.at[14,'building_name']=('The Club at Brickell Bay')
df.at[15,'building_name']=('7400 Oceanside at Fisher Island')
df.at[18,'building_name']=('350 Meridian')
df.at[24,'building_name']=('Blue Diamond')
df.at[34,'building_name']=('Key Colony')
df.at[41,'building_name']=('Reach Condo')



df.at[14,'address_only']=('1200 Brickell Bay Dr 3901 Miami')
df.at[15,'address_only']=('7431 Fisher Island Dr 7431 Miami Beach')
df.at[18,'address_only']=('Meridian Ave PH Miami Beach')
df.at[24,'address_only']=('4779 Collins Ave 2408 Miami Beach')
df.at[34,'address_only']=('201 Crandon Blvd 1028 Key Biscayne')
df.at[41,'address_only']=('68 SE 6th St 3103')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [26]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'November 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [27]:
m.save('index.html')

In [ ]:
### Clean DF for graphic ###

In [28]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/2911951518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/2911951518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [29]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/310194361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/310194361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/310194361.py:4: SettingWithCopyWarning: 
A value i

In [30]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/879808360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/879808360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [31]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/1544171665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_on_market'] = pd.to_numeric(df['days_on_market'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24881/1544171665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])


In [32]:
df.sort_values('Sale Price int',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
0,Estates at Aqualina\r17901 Collins Ave 4207\rSunny Isles Beach,6/30/2022,11/1/2022,42,"$11,000,000.00",5573,"$1,973.80",Marla Cohen,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Aqualina 17901 Collins Ave 4207 Sunny Isles Beach,Estates at Aqualina,17901 Collins Ave 4207 Sunny Isles Beach,"(17901 Collins Ave #4207, Sunny Isles Beach, FL 33160, USA, (25.9420489, -80.12019839999999))","(25.9420489, -80.12019839999999, 0.0)",25.942049,-80.120198,0.0,11000000.0,1973.80
15,7400 Oceanside at Fisher Island\r7431 Fisher Island Dr 7431\rMiami Beach,NaN,11/14/2022,120,"$9,250,000.00",5679,"$2,263.83",Dora Puig,Luxe Living Realty,Maria Oneide Gioia,Luxury Living Realty,7400 Oceanside at Fisher Island 7431 Fisher Island Dr 7431 Miami Beach,7400 Oceanside at Fisher Island,7431 Fisher Island Dr 7431 Miami Beach,"(Fisher Island, FL 33109, USA, (25.7609329, -80.14004589999999))","(25.7609329, -80.14004589999999, 0.0)",25.760933,-80.140046,0.0,9250000.0,2263.83
35,The Gables Club\r60 Edgewater Dr TS-A\rCoral Gables,NaN,11/30/2022,622,"$9,250,000.00",10190,$907.75,Dennis Carvajal,One Sotheby's Int'l Realty,Saddy Delgado PA,One Sotheby's Int'l Realty,The Gables Club 60 Edgewater Dr TS-A Coral Gables,The Gables Club,60 Edgewater Dr TS-A Coral Gables,"(60 Edgewater Dr, Coral Gables, FL 33133, USA, (25.706287, -80.2532669))","(25.706287, -80.2532669, 0.0)",25.706287,-80.253267,0.0,9250000.0,907.75
25,Estates at Acqualina\r17901 Collins Ave 1104\rSunny Isles Beach,NaN,11/21/2022,51,"$7,823,000.00",4130,"$1,894.19",Michael Goldstein,Acqualina Realty,Gil Brendel,"G Realty Group, LLC.",Estates at Acqualina 17901 Collins Ave 1104 Sunny Isles Beach,Estates at Acqualina,17901 Collins Ave 1104 Sunny Isles Beach,"(17901 Collins Ave #1104, Sunny Isles Beach, FL 33160, USA, (25.9419666, -80.1210341))","(25.9419666, -80.1210341, 0.0)",25.941967,-80.121034,0.0,7823000.0,1894.19
26,One Thousand Museum\r1000 Biscayne Blvd 3101\rMiami,NaN,11/22/2022,460,"$7,200,000.00",4600,"$1,565.22",Lana Bell,Douglas Elliman,Michael Duchon,The Corcoran Group,One Thousand Museum 1000 Biscayne Blvd 3101 Miami,One Thousand Museum,1000 Biscayne Blvd 3101 Miami,"(1000 Biscayne Blvd #3101, Miami, FL 33132, USA, (25.784171, -80.1900783))","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,7200000.0,1565.22
36,Residences by Armani Casa\r18975 Collins Ave + Cabana 3600\rSunny Isles Beach,NaN,11/30/2022,48,"$6,600,000.00",3315,"$1,990.95",Leonardo Villate Rodriguez,477 International Realty,Jose Antonio Correa Chacon,"Compass Florida, LLC",Residences by Armani Casa 18975 Collins Ave + Cabana 3600 Sunny Isles Beach,Residences by Armani Casa,18975 Collins Ave + Cabana 3600 Sunny Isles Beach,"(18975 Collins Ave, Sunny Isles Beach, FL 33160, USA, (25.9521284, -80.11970810000001))","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0,6600000.0,1990.95
16,Azure Condominium\r9401 Collins Ave 607\rSurfside,NaN,11/15/2022,211,"$6,500,000.00",4965,"$1,309.16",Silvia Coltrane,The Corcoran Group,Silvia Coltrane,The Corcoran Group,Azure Condominium 9401 Collins Ave 607 Surfside,Azure Condominium,9401 Collins Ave 607 Surfside,"(9401 Collins Ave #607, Surfside, FL 33154, USA, (25.8838004, -80.12177080000001))","(25.8838004, -80.12177080000001, 0.0)",25.883800,-80.121771,0.0,6500000.0,1309.16
17,Mansions at Acqualina\r17749 Collins Ave 701\rSunny Isles Beach,NaN,11/18/2022,75,"$6,400,000.00",4609,"$1,388.59",Richard Goihman,Douglas Elliman,Ryan Mendell,Maxwell E Realty Inc,Mansions at Acqualina 17749 Collins Ave 701 Sunny Isles Beach,Mansions at Acqualina,17749 Collins Ave 701 Sunny Isles Beach,"(17749 Collins Ave APT 701, Sunny Isles Beach, FL 33160, USA, (25.9400518, -80.12056079999999))","(25.9400518, -80.12056079999999, 0.0

In [47]:
df.sort_values('days_on_market',ascending=True)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
41,0Reach Condo\r68 SE 6th St 3103\rMiami,NaN,11/30/2022,3,"$1,800,000.00",1891,$951.88,Juliana Savoia PA,Coldwell Banker Realty,Alexandar Alexandrov,"Compass Florida, LLC",0Reach Condo 68 SE 6th St 3103 Miami,Reach Condo,68 SE 6th St 3103,"(68 SE 6th St Apt 3103, Miami, FL 33131, USA, (25.7680813, -80.1926812))","(25.7680813, -80.1926812, 0.0)",25.768081,-80.192681,0.0,1800000.0,951.88
2,Jade Beach\r17001 Collins Ave 1704\rSunny Isles Beach,10/10/2022,11/2/2022,6,"$2,450,000.00",1927,"$1,271.41",Marcello Agostini,"AG Real Estate Advisors, LLC",Marina Petlyakova,Golden Keys Inc,Jade Beach 17001 Collins Ave 1704 Sunny Isles Beach,Jade Beach,17001 Collins Ave 1704 Sunny Isles Beach,"(17001 Collins Ave APT 1704, Sunny Isles Beach, FL 33160, USA, (25.9335244, -80.1211641))","(25.9335244, -80.1211641, 0.0)",25.933524,-80.121164,0.0,2450000.0,1271.41
39,Sands of Key Biscayne Condo\r615 Ocean Dr 6A\rKey Biscayne,NaN,11/29/2022,8,"$3,000,000.00",1772,"$1,693.00",Juan Zuluaga,Graber Realty Group LLC,Maria Atalaya,Home-C International Realty LLC,Sands of Key Biscayne Condo 615 Ocean Dr 6A Key Biscayne,Sands of Key Biscayne Condo,615 Ocean Dr 6A Key Biscayne,"(615 Ocean Dr #6a, Miami, FL 33149, USA, (25.6901247, -80.1574734))","(25.6901247, -80.1574734, 0.0)",25.690125,-80.157473,0.0,3000000.0,1693.00
10,SLS Lux Brickell Residence\r801 S Miami Ave 3001\rMiami,NaN,11/7/2022,8,"$1,400,000.00",1613,$867.95,Karen Vieira Ferreira,Miami Realty Solution Group,Mariela Feo La Cruz,Realty One Group Evolution,SLS Lux Brickell Residence 801 S Miami Ave 3001 Miami,SLS Lux Brickell Residence,801 S Miami Ave 3001 Miami,"(801 S Miami Ave Unit 3001, Miami, FL 33130, USA, (25.7660758, -80.1929055))","(25.7660758, -80.1929055, 0.0)",25.766076,-80.192905,0.0,1400000.0,867.95
3,Majestic\r9601 Collins Ave 301\rBal Harbour,9/29/2022,11/4/2022,11,"$2,000,000.00",2260,$884.96,Shneur Shapira,The Brokerage South Florida Real Estate,Joel Lusky,The Brokerage South Florida Real Estate,Majestic 9601 Collins Ave 301 Bal Harbour,Majestic,9601 Collins Ave 301 Bal Harbour,"(9601 Collins Ave APT 301, Bal Harbour, FL 33154, USA, (25.8876563, -80.1222319))","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0,2000000.0,884.96
33,Biscayne Beach\r2900 NE 7th Ave 3702\rMiami,NaN,11/21/2022,32,"$1,600,000.00",1698,$942.29,Bento Queiroz,"Compass Florida, LLC",Alison Zhuk,Douglas Elliman,Biscayne Beach 2900 NE 7th Ave 3702 Miami,Biscayne Beach,2900 NE 7th Ave 3702 Miami,"(2900 NE 7th Ave Unit 3702, Miami, FL 33137, USA, (25.8048485, -80.18598399999999))","(25.8048485, -80.18598399999999, 0.0)",25.804848,-80.185984,0.0,1600000.0,942.29
13,Key Colony No 4 Condo\r141 Crandon Blvd 240\rKey Biscayne,NaN,11/8/2022,32,"$1,000,000.00",1715,$583.09,Pity Betancourt,BHHS EWM Realty,Pedro Mingo,Unique International Properties,Key Colony No 4 Condo 141 Crandon Blvd 240 Key Biscayne,Key Colony No,4 Condo 141 Crandon Blvd 240 Key Biscayne,"(141 Crandon Blvd APT 240, Key Biscayne, FL 33149, USA, (25.6977458, -80.16075560000002))","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0,1000000.0,583.09
14,0The Club at Brickell Bay\r1200 Brickell Bay Dr 3901\rMiami,NaN,11/7/2022,36,"$1,000,000.00",1105,$904.98,Jessica Bechard,Homeland Capital Realty,Yesenia Garcia,Homeland Capital Realty,0The Club at Brickell Bay 1200 Brickell Bay Dr 3901 Miami,The Club at Brickell Bay,1200 Brickell Bay Dr 3901 Miami,"(1200 Brickell Bay Dr APT 3901, Miami, FL 33131, USA, (25.7618036, -80.19058869999999))","(25.7618036, -80.19058869999999, 0.0)",25.761804,-80.190589,0.0,1000000.0,904.98
11,Parque Tower\r330 Sunny Isles Blvd Water View\rSunny Isles Beach,NaN,11/8/2022,38,"$1,100,000.00",1524,$721.78,Heloisa Arazi,AMG International Realty,Patricia Hernandez,AMG Internat

### Sale price sum

In [34]:
"${:,.2f}".format(df['Sale Price int'].sum())

'$143,375,500.00'

### Sale price average

In [35]:
"${:,.2f}".format(df['Sale Price int'].mean())

'$3,413,702.38'

### Price per sf average

In [42]:
"${:,.2f}".format(df['price_per_sqft_int'].mean())

'$1,190.67'

### Price per square foot min vs. max

In [40]:
"${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

'$2,263.83'

In [41]:
"${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

'$583.09'

### Sale price min and max

In [37]:
"${:,.2f}".format(df['Sale Price int'].max(axis=0))

'$11,000,000.00'

In [38]:
"${:,.2f}".format(df['Sale Price int'].min(axis=0))

'$1,000,000.00'

In [39]:
len(df)

42

In [ ]:
# df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
# df = df.replace('\n',' ',regex=True)
# df

## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [44]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesNovember2022
